In [1]:
%cd ..
%cd ..

/home/lingze/embedding_fusion/scripts
/home/lingze/embedding_fusion


In [2]:

from tqdm import tqdm
import numpy as np
import pandas as pd
import torch
import pickle
import os


from utils.data import DatabaseFactory

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

✓ Task extensions applied: BaseTask.mask_features


/home/lingze/anaconda3/envs/deepdb/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset = DatabaseFactory.get_dataset("hm")

In [4]:
db = DatabaseFactory.get_db("hm")

Loading Database object from /home/lingze/.cache/relbench/rel-hm/db...
Done in 2.22 seconds.


In [5]:
from utils.preprocess import infer_type_in_db

col_type_dict = infer_type_in_db(db, True)

[rule 0]: customer Inferred customer_id from numerical as categorical
[rule 1]: customer Inferred age from numerical as categorical
[rule 0]: customer Inferred postal_code from text_embedded as categorical
[rule 0]: article Inferred article_id from numerical as categorical
[rule 0]: article Inferred product_code from numerical as categorical
[rule 0]: article Inferred product_type_no from numerical as categorical
[rule 1]: article Inferred product_group_name from text_embedded as categorical
[rule 1]: article Inferred graphical_appearance_no from numerical as categorical
[rule 1]: article Inferred graphical_appearance_name from text_embedded as categorical
[rule 0]: article Inferred colour_group_code from numerical as categorical
[rule 1]: article Inferred colour_group_name from text_embedded as categorical
[rule 1]: article Inferred perceived_colour_value_id from numerical as categorical
[rule 1]: article Inferred perceived_colour_value_name from text_embedded as categorical
[rule 0]:

In [6]:
from utils.resource import get_text_embedder_cfg
text_embedder_cfg = get_text_embedder_cfg(
    # model_name = "sentence-transformers/average_word_embeddings_glove.6B.300d", 
    model_name = "all-MiniLM-L12-v2",
    device = device)

In [7]:
# materialize the tensor_frame 
from utils.builder import build_pyg_hetero_graph
cache_dir = "/home/lingze/embedding_fusion/data/hm-tensor-frame"
data, col_stats_dict = build_pyg_hetero_graph(
    db,
    col_type_dict,
    text_embedder_cfg,
    cache_dir,
    True,
)

-----> Materialize customer Tensor Frame
-----> Materialize article Tensor Frame
-----> Materialize transactions Tensor Frame
Build pyg hetero graph takes 5.692497 seconds


In [8]:
# save the col_type_dict
with open(os.path.join(cache_dir, "col_type_dict.pkl"), "wb") as f:
    pickle.dump(col_type_dict, f)